# Regridding using the `grid-weights` library

In [ ]:
import astropy.coordinates
import cdshealpix.nested
import cf_xarray  # noqa: F401
import geoarrow.rust.core as geoarrow
import grid_weights.api as grid_weights
import numpy as np
import xarray as xr
import xdggs  # noqa: F401
from grid_indexing import infer_cell_geometries

xr.set_options(keep_attrs=True, display_expand_attrs=False, display_expand_data=False)

In [ ]:
from distributed import Client

client = Client()
client

## rectilinear grid: the `air_temperature` example dataset

In [ ]:
ds = xr.tutorial.open_dataset("air_temperature", chunks={"time": 20}).isel(
    time=slice(None, 400)
)
ds

In [ ]:
upscaled = (
    ds.interp(lon=np.linspace(200, 330, 1060), lat=np.linspace(15, 76, 500))
    .assign_coords(lon=lambda ds: (ds["lon"] + 180) % 360 - 180)
    .chunk({"lon": 530, "lat": 250})
)
upscaled

In [ ]:
level = 7
lon = astropy.coordinates.Longitude(
    [200, 225, 250, 275, 300, 330, 330, 300, 275, 250, 225, 200], unit="degree"
)
lat = astropy.coordinates.Latitude(
    [15, 15, 15, 15, 15, 15, 75, 75, 75, 75, 75, 75], unit="degree"
)
cell_ids, _, _ = cdshealpix.nested.polygon_search(lon, lat, depth=level, flat=True)

target_grid = (
    xr.Dataset(coords={"cell_ids": ("cells", cell_ids)})
    .dggs.decode({"grid_name": "healpix", "level": level, "indexing_scheme": "nested"})
    .dggs.assign_latlon_coords()
)
target_grid

In [ ]:
source_geoms_ = geoarrow.to_shapely(infer_cell_geometries(upscaled)).reshape(
    (1060, 500)
)
source_geoms = xr.DataArray(
    source_geoms_, dims=["lon", "lat"], coords=upscaled[["lon", "lat"]].coords
).chunk({"lon": 530, "lat": 250})
source_geoms

In [ ]:
target_geoms = target_grid.dggs.cell_boundaries().chunk({"cells": 5100}).dggs.decode()
target_geoms

In [ ]:
%%time
algorithms = grid_weights.Algorithms.by_variable(upscaled, default="conservative")
indexed_cells = grid_weights.create_index(source_geoms).query(
    target_geoms, methods=algorithms.unique()
)
weights = grid_weights.weights(source_geoms, target_geoms, indexed_cells)
weights

In [ ]:
%%time
regridded = algorithms.regrid(upscaled, weights)
regridded

In [ ]:
%%time
computed = regridded.compute()
computed

In [ ]:
weights_ = weights.compute()
weights_

In [ ]:
sparse = (
    weights_.rename_dims({"target_cells": "cells"})
    .rename_vars({"target_cell_ids": "cell_ids"})
    .sel(cell_ids=46578)
)
dense = sparse.assign(
    conservative=lambda ds: ds["conservative"].copy(
        data=ds["conservative"].data.todense()
    )
)

display(sparse, dense)

In [ ]:
computed["air"].dggs.explore(alpha=0.8)